In [105]:
import random
import itertools
import numpy as np

In [106]:
def generate_game(bigNumbers) -> tuple[list[int], int]:
    if bigNumbers > 4 or bigNumbers < 0:
        raise ValueError("Invalid bigNumbers")

    largeNumbers = random.sample([25, 50, 75, 100], bigNumbers)
    smallNumbers = random.sample(
        list(range(1, 11)) + list(range(1, 11)), 6 - bigNumbers
    )

    numbers = largeNumbers + smallNumbers

    return (numbers, random.randint(101, 999))

In [107]:
operations = {
    "+": lambda x, y: x + y,
    "-": lambda x, y: x - y,
    "*": lambda x, y: x * y,
    "/": lambda x, y: x // y,  # integer division
}


def recusive_solve_countdown(numbers, target, calculations=[]):

    solutions = []

    # iterate over all pair of number pairs
    for pair in itertools.combinations(numbers, 2):

        # Remove the pair from the list of numbers and make copy
        computedNumbers = numbers.copy()
        computedNumbers.remove(pair[0])
        computedNumbers.remove(pair[1])

        for op, calculate in operations.items():
            a = max(pair)
            b = min(pair)

            if (a == 0 or b == 0) or (a % b != 0) and op == "/":
                continue  # skip division by zero and none whole number division

            val = calculate(a, b)

            calcs = [*calculations, (a, op, b, val)]

            if val == target:
                solutions.append(calcs)

            sols = recusive_solve_countdown([val, *computedNumbers], target, calcs)
            if len(sols) != 0:
                solutions = [*sols, *solutions]

    return solutions

In [108]:
def math_expression(expr, expresions):
    left = expresions.get(expr[0])

    if left is None:
        left = expr[0]
    else:
        left = math_expression(left, expresions)

    right = expresions.get(expr[2])
    if right is None:
        right = expr[2]
    else:
        right = math_expression(right, expresions)

    return f"({left} {expr[1]} {right})"

In [109]:
game = generate_game(2)

testGame = ([25, 100, 9, 7], 127)

solutions = recusive_solve_countdown(testGame[0], testGame[1])

print(len(solutions))
print()

for solution in solutions:
    for expr in solution:
        print(f"{expr[0]} {expr[1]} {expr[2]} = {expr[3]}")

    expressions = {}

    for expr in reversed(solution):
        expressions[expr[3]] = (expr[0], expr[1], expr[2])
    # recusivly expand the final expression
    print(math_expression(solution[-1], expressions))
    print()

18

9 - 7 = 2
100 + 25 = 125
125 + 2 = 127
((100 + 25) + (9 - 7))

9 - 7 = 2
100 + 2 = 102
102 + 25 = 127
((100 + (9 - 7)) + 25)

9 - 7 = 2
25 + 2 = 27
100 + 27 = 127
(100 + (25 + (9 - 7)))

100 - 7 = 93
25 + 9 = 34
93 + 34 = 127
((100 - 7) + (25 + 9))

100 - 7 = 93
93 + 9 = 102
102 + 25 = 127
(((100 - 7) + 9) + 25)

100 - 7 = 93
93 + 25 = 118
118 + 9 = 127
(((100 - 7) + 25) + 9)

100 + 9 = 109
25 - 7 = 18
109 + 18 = 127
((100 + 9) + (25 - 7))

100 + 9 = 109
109 - 7 = 102
102 + 25 = 127
(((100 + 9) - 7) + 25)

100 + 9 = 109
109 + 25 = 134
134 - 7 = 127
(((100 + 9) + 25) - 7)

25 - 7 = 18
100 + 9 = 109
109 + 18 = 127
((100 + 9) + (25 - 7))

25 - 7 = 18
18 + 9 = 27
100 + 27 = 127
(100 + ((25 - 7) + 9))

25 - 7 = 18
100 + 18 = 118
118 + 9 = 127
((100 + (25 - 7)) + 9)

25 + 9 = 34
100 - 7 = 93
93 + 34 = 127
((100 - 7) + (25 + 9))

25 + 9 = 34
34 - 7 = 27
100 + 27 = 127
(100 + ((25 + 9) - 7))

25 + 9 = 34
100 + 34 = 134
134 - 7 = 127
((100 + (25 + 9)) - 7)

100 + 25 = 125
9 - 7 = 2
125 + 2 